In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('20201103__ms__general__precinct.csv')
df = df.replace(np.nan, '', regex = True)

In [15]:
# adding change to fix near-duplicates due to scraping errors
df.loc[(df['precinct'].str.upper()=='MT ISREAL BAPTIST CHURCH')&(df.drop(columns = 'votes').duplicated()),'precinct']='MULTIPURPOSE CENTER'
df.loc[(df['precinct'].str.upper()=='PALMER')&(df.drop(columns = 'votes').duplicated()),'precinct']='PEOPLES'

In [16]:
df['office'] = df['office'].replace({'President': 'US PRESIDENT',
                                          'U.S. Senate': 'US SENATE',
                                           'U.S. House': 'US HOUSE',
                                           'US Senate': 'US SENATE',
                                          })

In [17]:
df['party_detailed'] = df['party'].replace({'D': 'DEMOCRAT',
                                            'R': 'REPUBLICAN',
                                            'AC': 'CONSTITUTION',
                                            'AS': 'AMERICAN SOLIDARITY',
                                            'IND': 'INDEPENDENT',
                                            'G': 'GREEN',
                                            'L': 'LIBERTARIAN',
                                            'I': 'INDEPENDENT',
                                            'DEM': 'DEMOCRAT',
                                            'REP': 'REPUBLICAN',
                                            'GRN': 'GREEN',
                                            'LIB': 'LIBERTARIAN'})

df['party_simplified'] = df['party_detailed'].replace({'GREEN': 'OTHER',
                                            'INDEPENDENT': 'OTHER',
                                            'AMERICAN SOLIDARITY': 'OTHER',
                                            'CONSTITUTION': 'OTHER'})

df = df.drop(['party'], axis=1)

In [18]:
df["candidate"] = df["candidate"].str.replace('.', '', regex=True)
df["candidate"] = df["candidate"].str.upper()

In [19]:
df['votes'] = df['votes'].replace({'x': '0'}) # ask what this is
df['votes'] = df['votes'].astype(int)

In [20]:
df['district'] = df['district'].astype(str)
df['district'] = df['district'].str.replace('\.0', '',regex=True)

In [21]:
def getDataverse(x):
    if 'US PRESIDENT' in x:
        return "PRESIDENT"
    elif "SENATE" in x:
        return "SENATE"
    elif "US HOUSE" in x:
        return "HOUSE"
    else: 
        return ''
    
df['dataverse'] = df['office'].apply(getDataverse)
df['dataverse'] = df['dataverse'].str.replace('\s+', ' ',regex=True)

In [22]:
df["state"] = 'MISSISSIPPI'
df["state_po"] = 'MS'
df["state_fips"] = '28'
df["state_cen"] = '64'
df["state_ic"] = '46'

In [23]:
df["stage"] = "GEN"
df["date"] = "2020-11-03"
df["year"] = "2020"

df["mode"] = "TOTAL"

df["special"] = "FALSE"
df["writein"] = "FALSE"
df['magnitude'] = 1

In [24]:
# After county name fix, append on fips codes
fips = pd.read_csv('../../../help-files/county-fips-codes.csv')

fips['state'] = fips['state'].str.upper()
fips = fips.applymap(str)
df['county_name'] = df['county'].str.upper()
df = df.drop(['county'], axis=1)
df=df.applymap(lambda x: x.strip() if type(x)==str else x)
df = pd.merge(df, fips, on = ['state','county_name'], 
             how = 'left')

In [25]:
df['jurisdiction_name'] = df['county_name']
df['jurisdiction_fips'] = df['county_fips']

In [26]:
df.loc[df['office'].str.contains('US SENATE') | df['office'].str.contains('US PRESIDENT'), 'district'] = 'STATEWIDE'

In [27]:
df.loc[df['district'].str.contains('1'), 'district'] = '001'
df.loc[df['district'].str.contains('2'), 'district'] = '002'
df.loc[df['district'].str.contains('3'), 'district'] = '003'
df.loc[df['district'].str.contains('4'), 'district'] = '004'

In [28]:
df['candidate'] = df['candidate'].replace({'ANTONIA ELLASON': 'ANTONIA ELIASON'})

In [29]:
df = df[~(df.precinct.str.contains("TOTAL"))]

In [30]:
#manual fixes from scraping errors

## US PRESIDENT
df.loc[(df['precinct']=='Springridge')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='HINDS'),
       'votes'] = 1553
df.loc[(df['precinct']=='Beat 5/Steele')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='SCOTT'),
       'votes'] = 324
df.loc[(df['precinct']=='Pace')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='BOLIVAR'),
       'votes'] = 228
df.loc[(df['precinct']=='Sanford')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COVINGTON'),
       'votes'] = 75
df.loc[(df['precinct']=='Riceville')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='AMITE'),
       'votes'] = 194
df.loc[(df['precinct']=='South Liberty')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='AMITE'),
       'votes'] = 102
df.loc[(df['precinct']=='Kellis Store')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='KEMPER'),
       'votes'] = 28

df.loc[(df['precinct']=='Lula')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 125
df.loc[(df['precinct']=='Cagle Crossing')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 21
df.loc[(df['precinct']=='Clarksdale # 3')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 374
df.loc[(df['precinct']=='Clarksdale # 4')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 1009
df.loc[(df['precinct']=='Clarksdale # 4 North')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 98
df.loc[(df['precinct']=='Clarksdale # 5')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 1021
df.loc[(df['precinct']=='Dublin')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 71
df.loc[(df['precinct']=='Sasse Street Fire Station')&(df['candidate']=='JOSEPH R BIDEN')&(df['county_name']=='COAHOMA'),
       'votes'] = 428

df.loc[(df['precinct']=='Dublin')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='COAHOMA'),
       'votes'] = 87
df.loc[(df['precinct']=='Clarksdale # 3')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='COAHOMA'),
       'votes'] = 70
df.loc[(df['precinct']=='Clarksdale # 5')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='COAHOMA'),
       'votes'] = 185
df.loc[(df['precinct']=='Clarksdale # 4 North')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='COAHOMA'),
       'votes'] = 59
df.loc[(df['precinct']=='Rena Lara')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='COAHOMA'),
       'votes'] = 181
df.loc[(df['precinct']=='Sasse Street Fire Station')&(df['candidate']=='DONALD J TRUMP')&(df['county_name']=='COAHOMA'),
       'votes'] = 21

df.loc[(df['precinct']=='Red House 55')&(df['candidate']=='BRIAN CARROLL')&(df['county_name']=='JEFFERSON DAVIS'),
       'votes'] = 0
df.loc[(df['precinct']=='Red House 55')&(df['candidate']=='DON BLANKENSHIP')&(df['county_name']=='JEFFERSON DAVIS'),
       'votes'] = 0
df.loc[(df['precinct']=='Precinct 54')&(df['candidate']=='KANYE WEST')&(df['county_name']=='HINDS'),
       'votes'] = 1
df.loc[(df['precinct']=='Crosby')&(df['candidate']=='KANYE WEST')&(df['county_name']=='AMITE'),
       'votes'] = 1

## US SENATE
df.loc[(df['precinct']=='Learned')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='HINDS'),
       'votes'] = 301
df.loc[(df['precinct']=='Beat 3 Coxburg (A-Z)')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='HOLMES'),
       'votes'] = 51
df.loc[(df['precinct']=='Belzoni South - 5bs 5010')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='HUMPHREYS'),
       'votes'] = 294
df.loc[(df['precinct']=='Dublin')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 77
df.loc[(df['precinct']=='Clarksdale # 3')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 58
df.loc[(df['precinct']=='Clarksdale # 4')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 46
df.loc[(df['precinct']=='Clarksdale # 4 North')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 55
df.loc[(df['precinct']=='Clarksdale # 5')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 150
df.loc[(df['precinct']=='Rena Lara')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 167
df.loc[(df['precinct']=='Sasse Street Fire Station')&(df['candidate']=='CINDY HYDE-SMITH')&(df['county_name']=='COAHOMA'),
       'votes'] = 8


df.loc[(df['precinct']=='Dublin')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 83
df.loc[(df['precinct']=='Cagle Crossing')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 22
df.loc[(df['precinct']=='Clarksdale # 3')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 383
df.loc[(df['precinct']=='Clarksdale # 4')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 1056
df.loc[(df['precinct']=='Clarksdale # 4 North')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 106
df.loc[(df['precinct']=='Lyon')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 371
df.loc[(df['precinct']=='Clarksdale # 5')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 1065
df.loc[(df['precinct']=='Sasse Street Fire Station')&(df['candidate']=='MIKE ESPY')&(df['county_name']=='COAHOMA'),
       'votes'] = 449

# US HOUSE
df.loc[(df['precinct']=='Cagle Crossing')&(df['candidate']=='BENNIE G THOMPSON')&(df['county_name']=='COAHOMA'),
       'votes'] = 18
df.loc[(df['precinct']=='Clarksdale # 3')&(df['candidate']=='BENNIE G THOMPSON')&(df['county_name']=='COAHOMA'),
       'votes'] = 370
df.loc[(df['precinct']=='Clarksdale # 4')&(df['candidate']=='BENNIE G THOMPSON')&(df['county_name']=='COAHOMA'),
       'votes'] = 1028
df.loc[(df['precinct']=='Clarksdale # 5')&(df['candidate']=='BENNIE G THOMPSON')&(df['county_name']=='COAHOMA'),
       'votes'] = 1028
df.loc[(df['precinct']=='Dublin')&(df['candidate']=='BENNIE G THOMPSON')&(df['county_name']=='COAHOMA'),
       'votes'] = 76
df.loc[(df['precinct']=='Sasse Street Fire Station')&(df['candidate']=='BENNIE G THOMPSON')&(df['county_name']=='COAHOMA'),
       'votes'] = 440

df.loc[(df['precinct']=='Dublin')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 81
df.loc[(df['precinct']=='Clarksdale # 3')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 59
df.loc[(df['precinct']=='Clarksdale # 4')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 52
df.loc[(df['precinct']=='Clarksdale # 4 North')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 52
df.loc[(df['precinct']=='Clarksdale # 5')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 161
df.loc[(df['precinct']=='Rena Lara')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 167
df.loc[(df['precinct']=='Sasse Street Fire Station')&(df['candidate']=='BRIAN FLOWERS')&(df['county_name']=='COAHOMA'),
       'votes'] = 10

df.loc[(df['precinct']=='Old Red Star')&(df['candidate']=='DOROTHY DOT BENFORD')&(df['county_name']=='LINCOLN'),
       'votes'] = 62


In [31]:
# Readme Check - No issues 
df["readme_check"] = "FALSE"
df['precinct'] = df['precinct'].str.upper().str.strip()

# Final step: Remove all trailing white space and put columns in correct order. 
df=df.applymap(lambda x: x.strip() if type(x)==str else x)

df=df[["precinct", "office", "party_detailed", "party_simplified", "mode", "votes", "county_name", "county_fips", "jurisdiction_name",
                      "jurisdiction_fips", "candidate", "district", "dataverse", "year", "stage", "state", "special", "writein", "state_po",
                      "state_fips", "state_cen", "state_ic", "date", "readme_check", "magnitude"]]

import csv
df.to_csv('../2020-ms-precinct-general.csv',quoting=csv.QUOTE_NONNUMERIC,index=False)

In [97]:
for x in df['precinct'].unique():
    if 'TOTAL' in x:
        print(x)

In [98]:
df

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
0,1 North Holly Springs,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,627,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
1,1 North Holly Springs,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,53,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
2,1 North Holly Springs,US PRESIDENT,CONSTITUTION,OTHER,TOTAL,1,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
3,1 North Holly Springs,US PRESIDENT,AMERICAN SOLIDARITY,OTHER,TOTAL,2,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
4,1 North Holly Springs,US PRESIDENT,INDEPENDENT,OTHER,TOTAL,2,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25848,Washington County Convention Center,US SENATE,DEMOCRAT,DEMOCRAT,TOTAL,665,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
25849,Washington County Convention Center,US SENATE,REPUBLICAN,REPUBLICAN,TOTAL,197,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
25850,Washington County Convention Center,US SENATE,LIBERTARIAN,LIBERTARIAN,TOTAL,4,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
25851,Washington County Convention Center,US HOUSE,REPUBLICAN,REPUBLICAN,TOTAL,202,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1


In [99]:
index = df[df['office'].str.contains('US SENATE') | df['office'].str.contains('US PRESIDENT')]
index

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
0,1 North Holly Springs,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,627,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
1,1 North Holly Springs,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,53,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
2,1 North Holly Springs,US PRESIDENT,CONSTITUTION,OTHER,TOTAL,1,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
3,1 North Holly Springs,US PRESIDENT,AMERICAN SOLIDARITY,OTHER,TOTAL,2,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
4,1 North Holly Springs,US PRESIDENT,INDEPENDENT,OTHER,TOTAL,2,MARSHALL,28093,MARSHALL,28093,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25846,Washington County Convention Center,US PRESIDENT,INDEPENDENT,OTHER,TOTAL,0,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
25847,Washington County Convention Center,US PRESIDENT,INDEPENDENT,OTHER,TOTAL,5,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
25848,Washington County Convention Center,US SENATE,DEMOCRAT,DEMOCRAT,TOTAL,665,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
25849,Washington County Convention Center,US SENATE,REPUBLICAN,REPUBLICAN,TOTAL,197,WASHINGTON,28151,WASHINGTON,28151,...,MISSISSIPPI,FALSE,FALSE,MS,28,64,46,2020-11-03,FALSE,1
